In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [5]:
%cd ./bias-bench
!python -m pip install -e .

/content/drive/MyDrive/bias-bench
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/bias-bench
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x39602000 @  0x7f6709b2f615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 1.9 kB/s 
     |████████████████████████████████| 3.5 MB 18.4 MB/s 
     |████████████████████████████████| 38.1 MB 1.1 MB/s 
     |████████████████████████████████| 1.5 MB 54.4 MB/s 
     |████████████████████████████████| 311 kB 74.8 MB/s 
     |████████████████████████████████| 58 kB 6.8 MB/s 
     |████████████████████████████████| 212 kB 71.8 MB/s 
 

In [6]:
import json
import pandas as pd
from collections import defaultdict

data_dir = './data/stereoset'

#run racial group experiment
res_dict = defaultdict(lambda: defaultdict(list))
race_combinations = [('hispanic', 'hispanic'), ('hispanic', 'white'), ('white', 'hispanic'), ('white', 'white'), ('white', 'black'), ('white', 'api'), ('black', 'white'), ('black', 'black'), ('api', 'white'), ('api', 'api'), ('merged', 'merged')]
for run in range(5):
    for male_race, female_race in race_combinations:
        !python experiments/stereoset.py --input_file "{data_dir}/gender_data_f_{female_race}_m_{male_race}_{run}.json" --model "GPT2LMHeadModel" --model_name_or_path "gpt2" --seed 1 --experiment_name 'f_{female_race}_m_{male_race}_{run}'
        !python experiments/stereoset_evaluation.py --predictions_file results/stereoset/stereoset_m-GPT2LMHeadModel_c-gpt2_s-1_f_{female_race}_m_{male_race}_{run}.json   --output_file results/stereoset/results_f_{female_race}_m_{male_race}_{run}.json --data_file "{data_dir}/gender_data_f_{female_race}_m_{male_race}_{run}.json"
        !python experiments/stereoset_debias.py --input_file "{data_dir}/gender_data_f_{female_race}_m_{male_race}_{run}.json" --model "SelfDebiasGPT2LMHeadModel" --model_name_or_path "gpt2" --seed 1 --experiment_name 'f_{female_race}_m_{male_race}_{run}_debias'
        !python experiments/stereoset_evaluation.py --predictions_file results/stereoset/stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1_f_{female_race}_m_{male_race}_{run}_debias.json   --output_file results/stereoset/results_f_{female_race}_m_{male_race}_{run}_debias.json --data_file "{data_dir}/gender_data_f_{female_race}_m_{male_race}_{run}.json"
        with open(f'./results/stereoset/results_f_{female_race}_m_{male_race}_{run}.json') as f:
            json_dict = json.load(f)
            res_dict['regular'][f'f_{female_race}_m_{male_race}'].append(float(list(json_dict.values())[0]['intrasentence']['implicit_gender']['SS Score']))
            
        with open(f'./results/stereoset/results_f_{female_race}_m_{male_race}_{run}_debias.json') as f:        
            json_dict = json.load(f)
            res_dict['debias'][f'f_{female_race}_m_{male_race}'].append(float(list(json_dict.values())[0]['intrasentence']['implicit_gender']['SS Score']))


Running StereoSet:
 - input_file: ./data/stereoset/gender_data_f_hispanic_m_hispanic_0.json
 - persistent_dir: /content/drive/MyDrive/bias-bench
 - model: GPT2LMHeadModel
 - model_name_or_path: gpt2
 - batch_size: 1
 - seed: 1
 - experiment_id : stereoset_m-GPT2LMHeadModel_c-gpt2_s-1
 - experiment_name : f_hispanic_m_hispanic_0
Downloading: 100% 665/665 [00:00<00:00, 524kB/s]
Downloading: 100% 523M/523M [00:08<00:00, 62.6MB/s]
Downloading: 100% 0.99M/0.99M [00:00<00:00, 2.04MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 1.08MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 2.64MB/s]
Using mask_token, but it is not set yet.
Evaluating intrasentence task.
100% 523/523 [00:10<00:00, 52.30it/s]
Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-GPT2LMHeadModel_c-gpt2_s-1_f_hispanic_m_hispanic_0.json
 - predictions_dir: None
 - output_file: results/stereoset/results_f_hispanic_m_hispanic_0.json
 - data_file: ./data/stereoset/gender_data_f_hispanic_m_hispanic_0.j

In [7]:
import numpy as np
from collections import defaultdict
import pandas as pd

#results dataframe
res_dict_mean = defaultdict(dict)
for key in res_dict.keys():
    for rc in res_dict[key].keys():
        res_dict_mean[key][rc] = np.mean(res_dict[key][rc])
df=pd.DataFrame(res_dict_mean)
df['delta']=abs(50-df['debias'])-abs(50-df['regular'])
df=df.round(2)
df

,regular,debias,delta
f_hispanic_m_hispanic,30.10,30.82,-0.73
f_white_m_hispanic,58.32,46.96,-5.28
f_hispanic_m_white,17.48,25.51,-8.03
f_white_m_white,58.51,46.81,-5.32
f_black_m_white,16.75,24.93,-8.18
f_api_m_white,42.91,41.41,1.49
f_white_m_black,63.40,56.79,-6.62
f_black_m_black,37.25,38.70,-1.45
f_white_m_api,58.47,54.03,-4.44
f_api_m_api,49.02,48.07,0.96
